In [46]:
# kaggle competitions download -c house-prices-advanced-regression-techniques -p "/Users/issacsmacbookpro/.cursor-tutor/projects/Housing Price Prediction"
# https://www.kaggle.com/code/ryannolan1/kaggle-housing-youtube-video

In [47]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from statsmodels.stats.outliers_influence import variance_inflation_factor
import xgboost as xgb

# Step 1: Data Import and Preliminary Inspection
train_df = pd.read_csv('/Users/issacsmacbookpro/.cursor-tutor/projects/Housing Price Prediction/house-prices-advanced-regression-techniques/train.csv')
test_df = pd.read_csv('/Users/issacsmacbookpro/.cursor-tutor/projects/Housing Price Prediction/house-prices-advanced-regression-techniques/test.csv')


## 1. Fill Missing Values 

In [48]:
# Assuming df is your dataframe
numeric_features_train = train_df.select_dtypes(include=['int64', 'float64'])

# Display the numeric features
print(numeric_features_train.columns)

# Check for missing values in numeric features
missing_values_train = numeric_features_train.isnull().sum()

# Display features with missing values
print(missing_values_train[missing_values_train > 0])

Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
       'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
       'MiscVal', 'MoSold', 'YrSold', 'SalePrice'],
      dtype='object')
LotFrontage    259
MasVnrArea       8
GarageYrBlt     81
dtype: int64


In [49]:
# Assuming df is your dataframe
numeric_features_test = test_df.select_dtypes(include=['int64', 'float64'])

# Display the numeric features
print(numeric_features_test.columns)

# Check for missing values in numeric features
missing_values_test = numeric_features_test.isnull().sum()

# Display features with missing values
print(missing_values_test[missing_values_test > 0])

Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
       'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
       'MiscVal', 'MoSold', 'YrSold'],
      dtype='object')
LotFrontage     227
MasVnrArea       15
BsmtFinSF1        1
BsmtFinSF2        1
BsmtUnfSF         1
TotalBsmtSF       1
BsmtFullBath      2
BsmtHalfBath      2
GarageYrBlt      78
GarageCars        1
GarageArea        1
dtype: int64


In [50]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Assuming df is your dataframe
# Step 1: Sort out numeric features
numeric_features = train_df.select_dtypes(include=['int64', 'float64'])

# Step 2: Exclude 'LotArea', 'SalePrice', and 'Id'
numeric_features_excluded = numeric_features.drop(columns=['Id'], errors='ignore')

# Step 3: Create an empty figure
fig = go.Figure()

# Step 4: Add a scatter plot for each feature and make them invisible initially
for feature in numeric_features_excluded.columns:
    fig.add_trace(go.Scatter(x=train_df[feature], y=train_df['SalePrice'], mode='markers', name=feature, visible=False))

# Step 5: Make the first feature visible
fig.data[0].visible = True

# Step 6: Create slider steps
slider_steps = [
    {'label': feature, 'method': 'update', 'args': [{'visible': [i == j for j in range(len(fig.data))]}, {'title': f'{feature} vs SalePrice', 'xaxis': {'title': feature}}]}
    for i, feature in enumerate(numeric_features_excluded.columns)
]

# Step 7: Add slider to the figure
fig.update_layout(
    sliders=[{
        'active': 0,
        'pad': {"t": 50},
        'steps': slider_steps
    }],
    title=' vs SalePrice (Excluding LotArea, SalePrice, and Id)',
    xaxis_title='Feature Value',  # This will be updated dynamically
    yaxis_title='SalePrice'
)

fig.show()

In [51]:
# 1. 填补缺失值,将缺失值替换为零
columns_to_fill = ['LotFrontage', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 
                   'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 
                   'BsmtHalfBath', 'GarageYrBlt', 'GarageCars', 'GarageArea']

for col in columns_to_fill:
    train_df[col] = train_df[col].fillna(0)  # Replace missing values with zero
    test_df[col] = test_df[col].fillna(0)    # Replace missing values with zero

## Remove Outliers

In [52]:
# Remove outliers before log transformations
outlier_conditions = (
    (train_df['LotFrontage'] > 300) |
    (train_df['LotArea'] > 100000) |
    (train_df['MasVnrArea'] > 1500) |
    (train_df['BsmtFinSF1'] > 5000) |
    (train_df['TotalBsmtSF'] > 6000) |
    (train_df['1stFlrSF'] > 4500) |
    (train_df['GrLivArea'] > 5500) |
    (train_df['OpenPorchSF'] > 500) |
    (train_df['EnclosedPorch'] > 500) |
    (train_df['3SsnPorch'] > 400) |
    (train_df['ScreenPorch'] > 400) |
    (train_df['GarageYrBlt'] > 2024) |
    (train_df['MiscVal'] > 80000)
)
train_df = train_df[~outlier_conditions].reset_index(drop=True)

## 2. Create Derived Variables for Both Train and Test Data and Remove Outliers for Train Data. 

In [53]:
# House Age: Number of years from YrBuilt
train_df['house_age'] = train_df['YrSold'] - train_df['YearBuilt']
test_df['house_age'] = test_df['YrSold'] - test_df['YearBuilt']

# Total Floor SF: Total surface area of floors (1st + 2nd)
train_df['totFlrSF'] = train_df['1stFlrSF'] + train_df['2ndFlrSF']
test_df['totFlrSF'] = test_df['1stFlrSF'] + test_df['2ndFlrSF']

# Total Bathroom: Sum of all bathrooms
train_df['tot_bath'] = train_df['FullBath'] + train_df['HalfBath'] + train_df['BsmtHalfBath'] + train_df['BsmtFullBath']
test_df['totbath'] = test_df['FullBath'] + test_df['HalfBath'] + test_df['BsmtHalfBath'] + test_df['BsmtFullBath']

# Total Area: Total above ground area + Total Basement SF + Total Floor SF
train_df['tot_area'] = train_df['GrLivArea'] + train_df['TotalBsmtSF'] + train_df['totFlrSF']
test_df['tot_area'] = test_df['GrLivArea'] + test_df['TotalBsmtSF'] + test_df['totFlrSF']

# Total Porch Area: Sum of porch areas
train_df['tot_porch'] = train_df['OpenPorchSF'] + train_df['EnclosedPorch'] + train_df['3SsnPorch'] + train_df['ScreenPorch']
test_df['tot_porch'] = test_df['OpenPorchSF'] + test_df['EnclosedPorch'] + test_df['3SsnPorch'] + test_df['ScreenPorch']

# Is House Remodeled: Flag if house has been remodeled
train_df['Remod'] = (train_df['YearBuilt'] != train_df['YearRemodAdd']).astype(int)
test_df['Remod'] = (test_df['YearBuilt'] != test_df['YearRemodAdd']).astype(int)

# Remodeled Age: Year sold - Remodel date
train_df['Remodel_Age'] = train_df['YrSold'] - train_df['YearRemodAdd']
test_df['Remodel_Age'] = test_df['YrSold'] - test_df['YearRemodAdd']



# Recession: 1 if YrSold is 2008, 2009, or 2010
train_df['Recession'] = train_df['YrSold'].apply(lambda x: 1 if x in [2008, 2009, 2010] else 0)
test_df['Recession'] = test_df['YrSold'].apply(lambda x: 1 if x in [2008, 2009, 2010] else 0)

# Check for invalid 'YrSold' and 'YearRemodAdd' values
invalid_remodel_age_rows = train_df[train_df['YrSold'] < train_df['YearRemodAdd']][['Id', 'YrSold', 'YearRemodAdd']]
if not invalid_remodel_age_rows.empty:
    print("Warning: 'YrSold' is less than 'YearRemodAdd' for the following rows:\n" + str(invalid_remodel_age_rows))

# Remove rows where house_age or Remodel_Age is negative
train_df = train_df[(train_df['house_age'] >= 0) & (train_df['Remodel_Age'] >= 0)]
test_df = test_df[(test_df['house_age'] >= 0) & (test_df['Remodel_Age'] >= 0)]




      Id  YrSold  YearRemodAdd
514  524    2007          2008


In [54]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Assuming df is your dataframe
# Step 1: Sort out numeric features
numeric_features = train_df.select_dtypes(include=['int64', 'float64'])

# Step 2: Exclude 'SalePrice' and 'Id'
numeric_features_excluded = numeric_features.drop(columns=['SalePrice', 'Id'], errors='ignore')

# Add derived variables to the list of features to check
derived_variables = ['house_age', 'totFlrSF', 'tot_bath', 'tot_area', 'tot_porch', 'Remod', 'Remodel_Age','Recession',
                    ]

# Step 3: Create an empty figure
fig = go.Figure()

# Step 4: Add a scatter plot for each derived feature and make them invisible initially
for feature in derived_variables:
    fig.add_trace(go.Scatter(x=train_df[feature], y=train_df['SalePrice'], mode='markers', name=feature, visible=False))

# Step 5: Make the first derived feature visible
fig.data[0].visible = True

# Step 6: Create slider steps for derived features
slider_steps = [
    {'label': feature, 'method': 'update', 'args': [{'visible': [i == j for j in range(len(fig.data))]}, {'title': f'Derived Features: {feature} vs SalePrice', 'xaxis': {'title': feature}}]}
    for i, feature in enumerate(derived_variables)
]


# Step 8: Add slider and dropdown to the figure
fig.update_layout(
    sliders=[{
        'active': 0,
        'pad': {"t": 50},
        'steps': slider_steps
    }],
    title='Derived Features vs SalePrice',
    xaxis_title='Feature Value',  # This will be updated dynamically
    yaxis_title='SalePrice'
)

fig.show()

In [55]:
# Remove outliers based on derived features before log transformations
outlier_conditions = (
    (train_df['totFlrSF'] > 4000) |   # Example condition for total floor area
    (train_df['tot_area'] > 10000) |  # Example condition for total area
    (train_df['tot_porch'] > 500)     # Example condition for total porch area
)  # Closing parenthesis added here
train_df = train_df[~outlier_conditions].reset_index(drop=True)

In [56]:
# Remove the variables used to create derived features from train_df
train_df = train_df.drop(columns=['Id'], errors='ignore')

# Remove derived feature variables from test_df
test_df = test_df.drop(columns=['Id'], errors='ignore')


In [57]:

# Step 1: Identify numeric features in the training set
numeric_features_train = train_df.select_dtypes(include=['int64', 'float64']).columns

# Display the numeric features
print("Numeric Features in Train Dataset:", numeric_features_train.tolist())

# Step 3: Convert 'MoSold' and 'YrSold' to categorical variables in both train and test datasets
categorical_columns = ['MoSold', 'YrSold']

for col in categorical_columns:
    train_df[col] = train_df[col].astype('category')
    test_df[col] = test_df[col].astype('category')

# Step 4: Identify all categorical variables by excluding numeric features
# This includes 'MoSold', 'YrSold', and any other non-numeric columns
categorical_features_train = train_df.select_dtypes(exclude=['int64', 'float64']).columns.tolist()
categorical_features_test = test_df.select_dtypes(exclude=['int64', 'float64']).columns.tolist()

# Alternatively, if you want to include specific dtypes like 'object' and 'category', you can use:
# categorical_features_train = train_df.select_dtypes(include=['object', 'category']).columns.tolist()
# categorical_features_test = test_df.select_dtypes(include=['object', 'category']).columns.tolist()

# Step 5: Display the identified categorical features
print("Categorical Variables in Train Dataset:", categorical_features_train)
print("Categorical Variables in Test Dataset:", categorical_features_test)


Numeric Features in Train Dataset: ['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice', 'house_age', 'totFlrSF', 'tot_bath', 'tot_area', 'tot_porch', 'Remod', 'Remodel_Age', 'Recession']
Categorical Variables in Train Dataset: ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposu

In [58]:

# Check if 'MoSold' and 'YrSold' are in categorical_features_train
if 'MoSold' in categorical_features_train:
    print("'MoSold' is in categorical_features_train.")
else:
    print("'MoSold' is NOT in categorical_features_train.")

if 'YrSold' in categorical_features_train:
    print("'YrSold' is in categorical_features_train.")
else:
    print("'YrSold' is NOT in categorical_features_train.")

'MoSold' is in categorical_features_train.
'YrSold' is in categorical_features_train.


## 3. Log Transformation For Numeric Variables


In [59]:
# Identify numeric variables in train_df and test_df
numeric_vars_train = train_df.select_dtypes(include=['int64', 'float64']).columns.tolist()
numeric_vars_test = test_df.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Log Transformations for remaining numeric variables
for var in numeric_vars_train:
    train_df[var] = train_df[var].replace(0, 1)  # Replace zeros with 1
    train_df[f'lg_{var}'] = np.log(train_df[var])  # Apply log transformation

for var in numeric_vars_test:
    test_df[var] = test_df[var].replace(0, 1)  # Replace zeros with 1
    test_df[f'lg_{var}'] = np.log(test_df[var])  # Apply log transformation

In [60]:
# Identify log-transformed variables
log_transformed_vars_train = [f'lg_{var}' for var in numeric_vars_train]
log_transformed_vars_test = [f'lg_{var}' for var in numeric_vars_test]

# Check for invalid values in log-transformed variables for train_df
invalid_log_vars_train = [var for var in log_transformed_vars_train if (train_df[var] < 0).any() or train_df[var].isnull().any()]

# Check for invalid values in log-transformed variables for test_df
invalid_log_vars_test = [var for var in log_transformed_vars_test if (test_df[var] < 0).any() or test_df[var].isnull().any()]

# Display the invalid log-transformed variables
print("Log-transformed variables with invalid values in train_df:")
print(invalid_log_vars_train)

print("Log-transformed variables with invalid values in test_df:")
print(invalid_log_vars_test)

Log-transformed variables with invalid values in train_df:
[]
Log-transformed variables with invalid values in test_df:
[]


In [61]:
# Lot Area and Frontage Interaction: Log-transformed
train_df['lg_ltarea_front'] = np.log((train_df['LotArea'] + 1) * (train_df['LotFrontage'] + 1))
test_df['lg_ltarea_front'] = np.log((test_df['LotArea'] + 1) * (test_df['LotFrontage'] + 1))

In [62]:
train_df.describe()


,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,lg_SalePrice,lg_house_age,lg_totFlrSF,lg_tot_bath,lg_tot_area,lg_tot_porch,lg_Remod,lg_Remodel_Age,lg_Recession,lg_ltarea_front
count,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,...,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,1440.0,1440.000000,1440.0,1440.000000
mean,56.961806,57.125000,9966.922222,6.081944,5.574306,1971.414583,1984.862500,101.094444,435.054861,46.204861,...,12.018738,2.939733,7.257210,0.801284,8.260212,3.076678,0.0,2.455388,0.0,12.671934
std,42.355366,32.668439,5336.634905,1.365293,1.107825,30.112223,20.665219,172.495048,429.386323,158.024753,...,0.392873,1.445718,0.326342,0.422071,0.301211,2.197735,0.0,1.389191,0.0,1.473212
min,20.000000,1.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,1.000000,1.000000,1.000000,...,10.471950,0.000000,5.811141,0.000000,6.504288,0.000000,0.0,0.000000,0.0,8.281471
25%,20.000000,41.750000,7500.000000,5.000000,5.000000,1954.000000,1967.000000,1.000000,1.000000,1.000000,...,11.773749,1.945910,7.021753,0.693147,8.060856,0.000000,0.0,1.386294,0.0,12.239394
50%,50.000000,63.000000,9423.000000,6.000000,5.000000,1973.000000,1994.000000,1.000000,378.000000,1.000000,...,11.995352,3.555348,7.283448,0.693147,8.273592,3.871201,0.0,2.639057,0.0,13.262376
75%,70.000000,78.000000,11475.750000,7.000000,6.000000,2001.000000,2004.000000,162.000000,705.250000,1.000000,...,12.269047,3.988984,7.477180,1.098612,8.464899,4.884689,0.0,3.713572,0.0,13.612890
max,190.000000,182.000000,70761.000000,10.000000,9.000000,2010.000000,2010.000000,1378.000000,2188.000000,1474.000000,...,13.345507,4.912655,8.196161,1.791759,9.175335,6.107023,0.0,4.094345,0.0,15.322136


In [63]:
test_df.describe

<bound method NDFrame.describe of       MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0             20       RH         80.0    11622   Pave   NaN      Reg   
1             20       RL         81.0    14267   Pave   NaN      IR1   
2             60       RL         74.0    13830   Pave   NaN      IR1   
3             60       RL         78.0     9978   Pave   NaN      IR1   
4            120       RL         43.0     5005   Pave   NaN      IR1   
...          ...      ...          ...      ...    ...   ...      ...   
1454         160       RM         21.0     1936   Pave   NaN      Reg   
1455         160       RM         21.0     1894   Pave   NaN      Reg   
1456          20       RL        160.0    20000   Pave   NaN      Reg   
1457          85       RL         62.0    10441   Pave   NaN      Reg   
1458          60       RL         74.0     9627   Pave   NaN      Reg   

     LandContour Utilities LotConfig  ... lg_MiscVal lg_house_age lg_totFlrSF  \
0       

## 4. Ordinal and nominal varaibles Encoding

In [64]:
# 3. 处理有序的分类变量，将其编码
ordinal_mappings = {
    'ExterQual': {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5, np.nan: 0},
    'ExterCond': {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5, np.nan: 0},
    'BsmtQual': {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5, np.nan: 0},
    'BsmtCond': {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5, np.nan: 0},
    'BsmtExposure': {'No': 1, 'Mn': 2, 'Av': 3, 'Gd': 4, np.nan: 0},
    'BsmtFinType1': {'Unf': 1, 'LwQ': 2, 'Rec': 3, 'BLQ': 4, 'ALQ': 5, 'GLQ': 6, np.nan: 0},
    'BsmtFinType2': {'Unf': 1, 'LwQ': 2, 'Rec': 3, 'BLQ': 4, 'ALQ': 5, 'GLQ': 6, np.nan: 0},
    'HeatingQC': {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5, np.nan: 0},
    'KitchenQual': {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5, np.nan: 0},
    'Functional': {
        'Sal': 1,
        'Sev': 2,
        'Maj2': 3,
        'Maj1': 4,
        'Mod': 5,
        'Min2': 6,
        'Min1': 7,
        'Typ': 8,
        np.nan: 0
    },
    'FireplaceQu': {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5, np.nan: 0},
    'GarageFinish': {'Unf': 1, 'RFn': 2, 'Fin': 3, np.nan: 0},
    'GarageQual': {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5, np.nan: 0},
    'GarageCond': {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5, np.nan: 0},
    'PoolQC': {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5, np.nan: 0},
    'Fence': {'MnWw': 1, 'MnPrv': 2, 'GdWo': 3, 'GdPrv': 4, np.nan: 0}
}

for col, mapping in ordinal_mappings.items():
    train_df[col] = train_df[col].map(mapping).fillna(0).astype(int)
    test_df[col] = test_df[col].map(mapping).fillna(0).astype(int)

In [65]:
# Identify all categorical variables in the train and test datasets

# Define the list of ordinal variables
ordinal_vars = [
    'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
    'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'KitchenQual',
    'Functional', 'FireplaceQu', 'GarageFinish', 'GarageQual', 'GarageCond',
    'PoolQC', 'Fence'
]

# Identify nominal variables by excluding ordinal variables from categorical variables
nominal_vars_train = [var for var in categorical_features_train if var not in ordinal_vars]
nominal_vars_test = [var for var in categorical_features_test if var not in ordinal_vars]

# Display the identified nominal variables
print("Nominal Variables in Train Dataset:", nominal_vars_train)
print("Nominal Variables in Test Dataset:", nominal_vars_test)

# Set categories for nominal variables before filling NaN values
for var in nominal_vars_train:
    # Ensure the column is of category dtype
    train_df[var] = train_df[var].astype('category')  # Convert to category if not already
    if 'None' not in train_df[var].cat.categories:  # Check if 'None' is already a category
        train_df[var] = train_df[var].cat.add_categories('None')  # Add 'None' as a category
    train_df[var] = train_df[var].fillna('None')  # Fill NaN with 'None'

for var in nominal_vars_test:
    # Ensure the column is of category dtype
    test_df[var] = test_df[var].astype('category')  # Convert to category if not already
    if 'None' not in test_df[var].cat.categories:  # Check if 'None' is already a category
        test_df[var] = test_df[var].cat.add_categories('None')  # Add 'None' as a category
    test_df[var] = test_df[var].fillna('None')  # Fill NaN with 'None'

Nominal Variables in Train Dataset: ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation', 'Heating', 'CentralAir', 'Electrical', 'GarageType', 'PavedDrive', 'MiscFeature', 'MoSold', 'YrSold', 'SaleType', 'SaleCondition']
Nominal Variables in Test Dataset: ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation', 'Heating', 'CentralAir', 'Electrical', 'GarageType', 'PavedDrive', 'MiscFeature', 'MoSold', 'YrSold', 'SaleType', 'SaleCondition']


In [66]:
# Check if 'YrSold' and 'MoSold' are in nominal_vars_train
if 'YrSold' in nominal_vars_train:
    print("'YrSold' is in nominal_vars_train.")
else:
    print("'YrSold' is NOT in nominal_vars_train.")

if 'MoSold' in nominal_vars_train:
    print("'MoSold' is in nominal_vars_train.")
else:
    print("'MoSold' is NOT in nominal_vars_train.")

'YrSold' is in nominal_vars_train.
'MoSold' is in nominal_vars_train.


In [67]:
# Check data types for columns in nominal_vars_train
print("Data types for nominal variables in Train Dataset:")
print(train_df[nominal_vars_train].dtypes)

# Check data types for columns in nominal_vars_test
print("\nData types for nominal variables in Test Dataset:")
print(test_df[nominal_vars_test].dtypes)

# Check if categorical variables in train_df are ready for one-hot encoding
print("Checking Train Dataset:")
for var in nominal_vars_train:
    print(f"Column: {var}, Type: {train_df[var].dtype}, Missing Values: {train_df[var].isnull().sum()}")

# Check if categorical variables in test_df are ready for one-hot encoding
print("\nChecking Test Dataset:")
for var in nominal_vars_test:
    print(f"Column: {var}, Type: {test_df[var].dtype}, Missing Values: {test_df[var].isnull().sum()}")

Data types for nominal variables in Train Dataset:
MSZoning         category
Street           category
Alley            category
LotShape         category
LandContour      category
Utilities        category
LotConfig        category
LandSlope        category
Neighborhood     category
Condition1       category
Condition2       category
BldgType         category
HouseStyle       category
RoofStyle        category
RoofMatl         category
Exterior1st      category
Exterior2nd      category
MasVnrType       category
Foundation       category
Heating          category
CentralAir       category
Electrical       category
GarageType       category
PavedDrive       category
MiscFeature      category
MoSold           category
YrSold           category
SaleType         category
SaleCondition    category
dtype: object

Data types for nominal variables in Test Dataset:
MSZoning         category
Street           category
Alley            category
LotShape         category
LandContour      category


In [68]:
# Identify categorical variables in the training and test datasets
categorical_features_train = train_df.select_dtypes(include=['object', 'category']).columns.tolist()
categorical_features_test = test_df.select_dtypes(include=['object', 'category']).columns.tolist()

# Proceed with One-Hot Encoding for all categorical variables
train_df_encoded = pd.get_dummies(train_df, columns=categorical_features_train, drop_first=True)
test_df_encoded = pd.get_dummies(test_df, columns=categorical_features_test, drop_first=True)

# Align the training and test datasets to ensure they have the same features
train_df_encoded, test_df_encoded = train_df_encoded.align(test_df_encoded, join='left', axis=1, fill_value=0)

In [69]:
print(train_df_encoded.shape)
print(test_df_encoded.shape)
test_df_encoded.head()

(1440, 294)
(1457, 294)


,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,...,SaleType_New,SaleType_Oth,SaleType_WD,SaleType_None,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,SaleCondition_None
0,20,80.0,11622,5,6,1961,1961,1.0,3,3,...,0,0,1,0,0,0,0,1,0,0
1,20,81.0,14267,6,6,1958,1958,108.0,3,3,...,0,0,1,0,0,0,0,1,0,0
2,60,74.0,13830,5,5,1997,1998,1.0,3,3,...,0,0,1,0,0,0,0,1,0,0
3,60,78.0,9978,6,6,1998,1998,20.0,3,3,...,0,0,1,0,0,0,0,1,0,0
4,120,43.0,5005,8,5,1992,1992,1.0,4,3,...,0,0,1,0,0,0,0,1,0,0


In [70]:
# Identify columns related to 'YrSold' and 'MoSold'
related_columns = [col for col in train_df_encoded.columns if 'YrSold' in col or 'MoSold' in col]

# Display the related columns
print("Columns related to 'YrSold' and 'MoSold':", related_columns)

Columns related to 'YrSold' and 'MoSold': ['MoSold_2', 'MoSold_3', 'MoSold_4', 'MoSold_5', 'MoSold_6', 'MoSold_7', 'MoSold_8', 'MoSold_9', 'MoSold_10', 'MoSold_11', 'MoSold_12', 'MoSold_None', 'YrSold_2007', 'YrSold_2008', 'YrSold_2009', 'YrSold_2010', 'YrSold_None']


In [71]:
# Check if the specified columns are in x_train_reduced
columns_to_check = ['YrSold', 'MoSold','house_age', 'Remodel_Age']
missing_columns = [col for col in columns_to_check if col not in train_df_encoded.columns]

if not missing_columns:
    print("All specified columns are present in train_df_encoded.")
else:
    print(f"The following columns are missing from train_df_encoded: {missing_columns}")



X_train = train_df_encoded.drop(columns=['lg_SalePrice'])
y_train = train_df_encoded['lg_SalePrice']


The following columns are missing from train_df_encoded: ['YrSold', 'MoSold']


In [72]:

# Step 7: Multicollinearity Analysis and Feature Reduction
# Correlation Analysis
# Drop target variable and aggregated columns from features


import pandas as pd

# Assuming X_train is your feature set
corr_matrix = X_train.corr().abs()

# Selecting the upper triangle of the correlation matrix
upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Identify features with correlation higher than threshold
threshold = 0.8
to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] > threshold)]

# Ensure 'YrSold', 'house_age', 'Remodel_Age' are not dropped
excluded_features = ['MoSold_2', 'MoSold_3', 'MoSold_4', 'MoSold_5', 'MoSold_6', 'MoSold_7', 'MoSold_8', 'MoSold_9', 
                     'MoSold_10', 'MoSold_11', 'MoSold_12',
                      'YrSold_2007', 'YrSold_2008', 'YrSold_2009', 'YrSold_2010', 'lg_house_age', 'lg_Remodel_Age']
to_drop = [feature for feature in to_drop if feature not in excluded_features]  # Exclude specified features

# Print features to be dropped
print("Features to be dropped due to high correlation:")
print(to_drop)

# Drop these features
X_train_reduced = X_train.drop(columns=to_drop)



from sklearn.ensemble import RandomForestRegressor

# ... existing code ...
# Train a simple Random Forest model to get feature importance
model = RandomForestRegressor()
model.fit(X_train, y_train)

# Get feature importances
feature_importances = pd.Series(model.feature_importances_, index=X_train.columns)
sorted_features = feature_importances.sort_values(ascending=False)

# Retain highly correlated features based on importance
for feature in upper_triangle.columns:
    if any(upper_triangle[feature] > threshold):
        correlated_features = upper_triangle.index[upper_triangle[feature] > threshold].tolist()
        for correlated_feature in correlated_features:
            # Check if the feature to drop is not in excluded features
            if feature_importances[feature] > feature_importances[correlated_feature] and correlated_feature not in excluded_features:  # {{ edit_2 }}
                to_drop.append(correlated_feature)  # Drop the less important feature
# ... existing code ...

# Print additional features to be dropped based on importance
print("Additional features to be dropped based on importance:")
print(set(to_drop))

X_train_reduced = X_train.drop(columns=set(to_drop))


Features to be dropped due to high correlation:
['1stFlrSF', 'TotRmsAbvGrd', 'GarageArea', 'GarageQual', 'GarageCond', 'PoolQC', 'SalePrice', 'house_age', 'totFlrSF', 'tot_area', 'Remodel_Age', 'lg_MSSubClass', 'lg_LotFrontage', 'lg_LotArea', 'lg_OverallQual', 'lg_OverallCond', 'lg_YearBuilt', 'lg_YearRemodAdd', 'lg_MasVnrArea', 'lg_BsmtFinSF1', 'lg_BsmtFinSF2', 'lg_TotalBsmtSF', 'lg_1stFlrSF', 'lg_2ndFlrSF', 'lg_LowQualFinSF', 'lg_GrLivArea', 'lg_BsmtFullBath', 'lg_BsmtHalfBath', 'lg_FullBath', 'lg_HalfBath', 'lg_BedroomAbvGr', 'lg_KitchenAbvGr', 'lg_TotRmsAbvGrd', 'lg_Fireplaces', 'lg_GarageYrBlt', 'lg_GarageCars', 'lg_GarageArea', 'lg_WoodDeckSF', 'lg_OpenPorchSF', 'lg_EnclosedPorch', 'lg_3SsnPorch', 'lg_ScreenPorch', 'lg_PoolArea', 'lg_totFlrSF', 'lg_tot_bath', 'lg_tot_area', 'lg_ltarea_front', 'MSZoning_RM', 'Neighborhood_Somerst', 'HouseStyle_1Story', 'HouseStyle_2Story', 'RoofStyle_Hip', 'Exterior2nd_CBlock', 'Exterior2nd_CmentBd', 'Exterior2nd_HdBoard', 'Exterior2nd_MetalSd', '

In [73]:
print(X_train_reduced.shape)
# Check if the specified columns are in x_train_reduced
columns_to_check = ['MoSold_2', 'MoSold_3', 'MoSold_4', 'MoSold_5', 'MoSold_6', 'MoSold_7', 'MoSold_8', 'MoSold_9', 
                     'MoSold_10', 'MoSold_11', 'MoSold_12',
                      'YrSold_2007', 'YrSold_2008', 'YrSold_2009', 'YrSold_2010', 'lg_house_age', 'lg_Remodel_Age']
missing_columns = [col for col in columns_to_check if col not in X_train_reduced.columns]

if not missing_columns:
    print("All specified columns are present in x_train_reduced.")
else:
    print(f"The following columns are missing from x_train_reduced: {missing_columns}")

(1440, 207)
All specified columns are present in x_train_reduced.


In [74]:
# Import necessary libraries
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
import time  # Import time module

# 7. Calculate Variance Inflation Factor (VIF)
def calculate_vif(df):
    vif_data = pd.DataFrame()
    vif_data['Feature'] = df.columns
    vif_data['VIF'] = [
        variance_inflation_factor(df.values, i) for i in range(df.shape[1])  # Ensure this returns a DataFrame
    ]
    return vif_data.sort_values(by='VIF', ascending=False)

def remove_high_vif_features(df, threshold=10, exclude_features=None, required_features=None):
    """
    Iteratively remove features with VIF > threshold, excluding specified features and retaining required features.
    """
    if exclude_features is None:
        exclude_features = []
    if required_features is None:
        required_features = []
        
    iteration = 1
    while True:
        start_time = time.time()  # Start timing the iteration
        vif = calculate_vif(df)
        # Sort VIF DataFrame by VIF values in descending order
        vif = vif.sort_values('VIF', ascending=False).reset_index(drop=True)
        
        # Find the feature with the highest VIF not in exclude_features or required_features
        for idx, row in vif.iterrows():
            if row['Feature'] not in exclude_features and row['Feature'] not in required_features:  # Check if feature is not in exclude lists
                max_vif = row['VIF']
                feature_to_remove = row['Feature']
                break
        else:
            # All remaining features are in exclude_features or required_features
            print("All remaining features are in exclude_features or required_features. Stopping iteration.")
            break
        
        print(f"Iteration {iteration}:")
        print(vif.head(10))
        print(f"Max VIF: {max_vif} (Feature to remove: {feature_to_remove})\n")
        
        if max_vif > threshold:
            print(f"Removing '{feature_to_remove}' with VIF: {max_vif}\n")
            df = df.drop(columns=[feature_to_remove])
            iteration += 1
        else:
            print("All VIF values are below the threshold or only excluded features have high VIF.\n")
            break
        
        end_time = time.time()  # End timing the iteration
        print(f"Time taken for iteration {iteration - 1}: {end_time - start_time:.4f} seconds\n")  # Print the time taken

    return df

# Ensure 'YrSold', 'house_age', 'Remodel_Age' are included in X_train_reduced
required_features = ['MoSold_2', 'MoSold_3', 'MoSold_4', 'MoSold_5', 'MoSold_6', 'MoSold_7', 'MoSold_8', 'MoSold_9', 
                     'MoSold_10', 'MoSold_11', 'MoSold_12',
                      'YrSold_2007', 'YrSold_2008', 'YrSold_2009', 'YrSold_2010', 'lg_house_age', 'lg_Remodel_Age']

# Keep only columns with more than one unique value, excluding required_features
unique_counts = X_train_reduced.nunique()
columns_to_keep = unique_counts[unique_counts > 1].index.tolist()

# Ensure required features are included
columns_to_keep += required_features
columns_to_keep = list(set(columns_to_keep))  # Remove duplicates if any
X_train_reduced = X_train_reduced.loc[:, columns_to_keep]

# Convert all columns to float64 type for VIF calculation
X_train_reduced = X_train_reduced.astype('float64')

# Use the correct column names in exclude_features
X_train_reduced = remove_high_vif_features(
    X_train_reduced,
    threshold=10,
    exclude_features=required_features
)

print(f"Shape after VIF reduction: {X_train_reduced.shape}")

final_vif = calculate_vif(X_train_reduced)
print("Final VIF Scores:\n", final_vif)

Iteration 1:
           Feature          VIF
0     Heating_GasA  1087.203106
1     BsmtHalfBath   864.420589
2  Condition2_Norm   787.122128
3      Street_Pave   461.207831
4         HalfBath   204.980348
5       Functional   186.178480
6     BsmtFullBath   166.491648
7        ExterQual   135.957785
8        ExterCond   112.601236
9      KitchenQual    88.480434
Max VIF: 1087.2031057224851 (Feature to remove: Heating_GasA)

Removing 'Heating_GasA' with VIF: 1087.2031057224851

Time taken for iteration 1: 17.2713 seconds

Iteration 2:
           Feature         VIF
0     BsmtHalfBath  775.394663
1  Condition2_Norm  708.682693
2      Street_Pave  442.310937
3         HalfBath  204.948479
4       Functional  182.813004
5     BsmtFullBath  164.718894
6        ExterQual  135.424255
7        ExterCond  111.559710
8      KitchenQual   88.450288
9     lg_house_age   77.010233
Max VIF: 775.3946632571174 (Feature to remove: BsmtHalfBath)

Removing 'BsmtHalfBath' with VIF: 775.3946632571174

Time

In [75]:
X_train_reduced.head()
X_train_reduced.to_csv('/Users/issacsmacbookpro/.cursor-tutor/projects/Housing Price Prediction/house-prices-advanced-regression-techniques/X_train_reduced.csv')
x_train = X_train_reduced
print(f"Shape after VIF reduction: {X_train_reduced.shape}")
x_train.shape

Shape after VIF reduction: (1440, 152)


(1440, 152)

In [76]:
x_train.head()
test_df.head()
y_train.head()

0    12.247694
1    12.109011
2    12.317167
3    11.849398
4    12.429216
Name: lg_SalePrice, dtype: float64

In [77]:
# ... existing code ...
# Align the training and test datasets to ensure they have the same features
x_train, test_df_encoded = train_df_encoded.align(test_df_encoded, join='left', axis=1, fill_value=0)

# Ensure test_df_encoded has the same columns as X_train_reduced
test_df = test_df_encoded.reindex(columns=X_train_reduced.columns, fill_value=0)  # {{ edit_1 }}
# ... existing code ...
print(X_train.shape)
print(test_df.shape)

(1440, 293)
(1457, 152)


## Using train_test_split to split the data into training and validation sets (80% train, 20% validation)

In [97]:
from sklearn.model_selection import train_test_split

# Split the data into training and validation sets (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42)


In [98]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

# Initialize the XGBoost Regressor
xgboost_model = xgb.XGBRegressor(
    n_estimators=5000,   # Number of trees (boosting rounds)
    learning_rate=0.001,  # Learning rate
    max_depth=6,         # Maximum depth of each tree
    subsample=0.8,       # Subsample ratio of the training instance
    colsample_bytree=0.8,  # Subsample ratio of columns when constructing each tree
    random_state=42
)

# Train the model
xgboost_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],  # Validation set for early stopping
    verbose=True  # Print training progress
)

[0]	validation_0-rmse:0.41327
[1]	validation_0-rmse:0.41292
[2]	validation_0-rmse:0.41258
[3]	validation_0-rmse:0.41227
[4]	validation_0-rmse:0.41199
[5]	validation_0-rmse:0.41166
[6]	validation_0-rmse:0.41132
[7]	validation_0-rmse:0.41104
[8]	validation_0-rmse:0.41071
[9]	validation_0-rmse:0.41036
[10]	validation_0-rmse:0.41003
[11]	validation_0-rmse:0.40979
[12]	validation_0-rmse:0.40945
[13]	validation_0-rmse:0.40912
[14]	validation_0-rmse:0.40878
[15]	validation_0-rmse:0.40847
[16]	validation_0-rmse:0.40814
[17]	validation_0-rmse:0.40781
[18]	validation_0-rmse:0.40748
[19]	validation_0-rmse:0.40715
[20]	validation_0-rmse:0.40688
[21]	validation_0-rmse:0.40655
[22]	validation_0-rmse:0.40621
[23]	validation_0-rmse:0.40587
[24]	validation_0-rmse:0.40555
[25]	validation_0-rmse:0.40522
[26]	validation_0-rmse:0.40490
[27]	validation_0-rmse:0.40460
[28]	validation_0-rmse:0.40427
[29]	validation_0-rmse:0.40400
[30]	validation_0-rmse:0.40375
[31]	validation_0-rmse:0.40342
[32]	validation_0-

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.001, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=5000, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [99]:
# Predict on the validation set
y_val_pred = xgboost_model.predict(X_val)

# Calculate RMSE
rmse = mean_squared_error(y_val, y_val_pred, squared=False)
print(f"Validation RMSE: {rmse:.4f}")


Validation RMSE: 0.0784
